In [19]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
from tqdm import tqdm

load_dotenv(".env")
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

word_data_file = open("word_data.json", "r")
word_data = json.loads(word_data_file.read())
word_data_file.close()

all_words = []
for segment in word_data["output"]["segments"]:
    all_words.extend(segment["words"])

word_list = []

for word in all_words:
    word_list.append(word["word"])

phonemes_set = set()
phonemes_map = {}

def split_list_into_k_parts(lst, k):
    return [lst[i * len(lst) // k: (i + 1) * len(lst) // k] for i in range(k)]

words_lists = split_list_into_k_parts(word_list, 60)

for wl in tqdm(words_lists):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "Given this list of words, output a JSON file where each word is mapped to a list of phonemes. This speech was delivered by President Barack Obama. Don't format the phonemes with slashes - just the representative characters themselves. Don't include punctuation or use any special symbols: the symbols used should be within the 44 standard symbols used to describe English pronunciation only. "
            },
            {
                "role": "user",
                "content": json.dumps(wl)
            }
        ],
        response_format={"type": "json_object"},
    )

    content = json.loads(response.choices[0].message.content)
    phonemes_map.update(content)

100%|██████████| 60/60 [08:29<00:00,  8.49s/it]


In [5]:
phonemes_set

set()

In [21]:
import string

all_words = []
for segment in word_data["output"]["segments"]:
    all_words.extend(segment["words"])

new_key_phonemes_map = {}
for key in phonemes_map:
    k = key.translate(str.maketrans('', '', string.punctuation)).lower()
    k = k.replace("-", "")
    k = k.replace("–", "")
    new_key_phonemes_map[k] = phonemes_map[key]

for i in range(len(word_list)):
    w = all_words[i]["word"].translate(str.maketrans('', '', string.punctuation)).lower()
    w = w.replace("-", "")
    w = w.replace("–", "")
    if len(w) == 0:
        continue
    print(w)
    all_words[i]["phonemes"] = new_key_phonemes_map[w]
    phonemes_set.update(new_key_phonemes_map[w])

print(list(phonemes_set))
print(len(list(phonemes_set)))

output_file = open("phoneme-data.json", "w+")
output_file.write(json.dumps(all_words, indent=4))
output_file.close()

it
meant
that
there
was
less
money
coming
in
and
it
required
us
to
spend
even
more
on
tax
cuts
for
middleclass
families
to
spur
the
economy
on
unemployment
insurance
on
aid
to
states
so
we
could
prevent
more
teachers
and
firefighters
and
police
officers
from
being
laid
off
these
emergency
steps
also
added
to
the
deficit
now
every
family
knows
a
little
credit
card
debt
is
manageable
but
if
we
stay
on
the
current
path
our
growing
debt
could
cost
us
jobs
and
do
serious
damage
to
the
economy
more
of
our
tax
dollars
will
go
toward
paying
off
the
interest
on
our
loans
businesses
will
be
less
likely
to
open
up
shop
and
hire
workers
in
a
country
that
cant
balance
its
books
interest
rates
could
climb
for
everyone
who
borrows
money
the
homeowner
with
a
mortgage
the
student
with
a
college
loan
the
corner
store
that
wants
to
expand
and
we
wont
have
enough
money
to
make
jobcreating
investments
in
things
like
education
and
infrastructure
or
pay
for
vital
programs
like
medicare
and
medicaid
because
n

KeyError: 'proposal'